In [3]:
#Instalando mesa con la librerías que se van a ocupar
!pip install numpy scipy matplotlib seaborn scikit-learn mesa==3.0 -q


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
fileref = open("ArchProj.txt", "r")
texto = fileref.readlines()
fileref.close()
grids = texto[:6]
victim = texto[6:15]
Fire = texto[15:24]
Door_lines = texto[24:37]
Access_point = texto[37:43]
print(grids)
print(victim)
print(Fire)
print(Door_lines)
print(Access_point)

['1011 1110 1011 1010 1000 1100 1001 1100\n', '1000 1000 1010 1110 0011 0110 0011 0110\n', '0011 0110 1101 1101 1111 1011 1110 1101\n', '1011 1010 0110 0111 1011 1010 1010 0110\n', '1001 1110 1001 1100 1101 1101 1101 1101\n', '0111 1011 0010 0110 0111 0111 0111 0111\n']
['1 1 f\n', '1 2 f\n', '1 8 v\n', '2 4 v\n', '3 5 v\n', '4 1 v\n', '6 2 v\n', '6 5 v\n', '6 7 v\n']
['1 3\n', '1 7\n', '2 3\n', '3 3\n', '5 2\n', '5 5\n', '5 7\n', '5 8\n', '6 3\n']
['1 2 1 3\n', '1 6 1 7\n', '2 4 2 5\n', '2 8 3 8\n', '3 3 3 4\n', '3 4 3 5\n', '3 5 3 6\n', '3 6 4 6\n', '4 4 5 4\n', '4 5 5 5\n', '4 6 5 6\n', '5 2 5 3\n', '6 6 6 7\n']
['1 4\n', '1 7\n', '2 1\n', '4 8\n', '6 4\n', '6 7\n']


In [5]:
#Conexion
import socket
import json
#Conexión
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import heapq, random
from os import read

Could not import SolaraViz. If you need it, install with 'pip install --pre mesa[viz]'


c:\Users\mbajo\OneDrive\Documentos\Tec\MULTIAGENTES-GRAFICASCOMP\Proyecto\multi_agent\Agentes\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:

# -------------------------------
# Leer txt y colocar en variables
# -------------------------------
fileref = open("ArchProj.txt", "r")
texto = fileref.readlines()
fileref.close()
grids = texto[:6]
victim = texto[6:15]
Fire = texto[15:24]
Door_lines = texto[24:37]
Access_point = texto[37:43]

# ------------------------------
# Parsear listas desde el archivo
# funciones realizada con ayuda de IA
# ------------------------------
def parse_positions(lines):
    coords = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 2:
            x, y = map(int, parts[:2])
            coords.append((x-1, y-1))
    return coords

def parse_positions_v_f(lines):
    coords = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 3:
            x, y = map(int, parts[:2])
            kind = parts[2]
            if kind == "v":
                coords.append((x-1, y-1, "v"))
            elif kind == "f":
                coords.append((x-1, y-1, "f"))


    return coords

def parse_doors(lines):
    coords = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 4:
            x1, y1, x2, y2 = map(int, parts[:4])
            coords.append((x1-1, y1-1, x2-1, y2-1))
    return coords

fire_coords = parse_positions(Fire)
door_coords = parse_doors(Door_lines)
access_coords = parse_positions(Access_point)
false_alarm_coords = parse_positions_v_f(victim)

In [60]:
# ------------------------------
# Valores para el mapa
# ------------------------------
EMPTY = 0
WALL = 1
VICTIM = 4
FALSE_ALARM = 5

###############################################################################

# ------------------------------
# Clase pared
# ------------------------------
class Cell:
  def __init__(self, up, left, down, right):
    self.positions = [up, left, down, right]
    self.damages = [1,1,1,1]

  def get_damages(self, side = None):
    if side == None:
      return self.damages
    else:
      return self.damages[side]

  def get_positions(self, side = None):
    if side != None:
      return self.positions[side]
    else:
      return self.positions

  def set_damage(self, side, damage):
    if self.damages[side] == 0:
      return
    else:
      self.damages[side] -= damage

  def is_empty(self):
      for position in self.positions:
        if position != EMPTY:
          return False
        else:
          return True

  def set_cell(self,values):
    if len(values) == 4:
      self.positions = values
    return self.positions

# ------------------------------
# Crear matriz de celdas 6x8 a
# partir de `grids`
# Función creada con ayuda de IA
# ------------------------------
height, width = 6, 8
matriz_pared = [[None for _ in range(width)] for _ in range(height)]

for i in range(height): # recorre las 6 fila
  fila = grids[i].strip().split() # separa en bloques (ej: '1011', '1110')
  for j in range(width): # recorre las 8 columnas
    valores = [int(x) for x in fila[j]] # convierte '1011' → [1,0,1,1]
    matriz_pared[i][j] = Cell(*valores) # desempaqueta en up,left,down,righ

for i in range(height):
  for j in range(width):
    print(matriz_pared[i][j].get_positions(), end=" ")
  print()

###############################################################################
# ------------------------------
# Clase Puertas
# ------------------------------
# 0 cerrada
# 1 abierta
# 2 destruida
class Door:
    CLOSED = 0
    OPEN = 1
    DESTROYED = 2

    def __init__(self, r1, c1, r2, c2):
        self.state = Door.CLOSED  # empieza cerrada
        self.cell1 = (r1, c1)
        self.cell2 = (r2, c2)

    def get_state(self):
        return self.state

    def set_state(self, state):
        self.state = state

    def open(self):
      if self.state == Door.CLOSED:
        self.state = Door.OPEN

    def close(self):
      if self.state == Door.OPEN:
        self.state = Door.CLOSED

    def destroy(self):
      self.state = Door.DESTROYED

    def connects(self, cell):
        # Devuelve la otra celda si cell es una de las conectadas por la puerta
        if cell == self.cell1:
            return self.cell2
        elif cell == self.cell2:
            return self.cell1
        return None

    def blocks_passage(self):
        """Devuelve True si la puerta está cerrada o destruida"""
        return self.state == Door.CLOSED  # cerrada = bloquea


###############################################################################
door_list = []
for (r1, c1, r2, c2) in door_coords:
    door = Door(r1, c1, r2, c2)
    door_list.append(door)

cell_doors = {}

for door in door_list:
    for cell in [door.cell1, door.cell2]:
        if cell not in cell_doors:
            cell_doors[cell] = []
        cell_doors[cell].append(door)

def open_door(access_coords):
    for (x,y) in access_coords:
        cell = (x, y)
        if cell in cell_doors:
            for door in cell_doors[cell]:   
                door.open()

print("Puertas por celda:")
for cell, doors in cell_doors.items():
    print(f"Celda {cell}: {len(doors)} puertas")

###############################################################################
#----------------------
# clase fire
#----------------------
class Fire:
    EMPTY = 0
    SMOKE = 1
    FIRE = 2
    EXPLODED = 3  # para marcar explosión

    def __init__(self, x, y):
        self.state = Fire.EMPTY
        self.x = x
        self.y = y
        self.to_explode = False  # bandera de explosión

    def get_state(self):
        return self.state

    def set_state(self, state):
        self.state = state

    def get_position(self):
        return (self.x, self.y)

    def get_neighbors(self, matriz_pared):
        neighbors = []
        directions = [(-1,0,0), (0,1,1), (1,0,2), (0,-1,3)]
        for dx, dy, side in directions:
            nx, ny = self.x + dx, self.y + dy
            if 0 <= nx < len(matriz_pared) and 0 <= ny < len(matriz_pared[0]):
                if matriz_pared[self.x][self.y].get_positions(side) == 0 and \
                   matriz_pared[nx][ny].get_positions((side + 2) % 4) == 0:
                    neighbors.append((nx, ny, side))
        return neighbors

    def set_smoke(self, matriz_fire, matriz_pared):
        neighbors = self.get_neighbors(matriz_pared)
        if neighbors:
            x, y, _ = random.choice(neighbors)
            if matriz_fire[x][y].get_state() == Fire.EMPTY:
                matriz_fire[x][y].set_state(Fire.SMOKE)

    def expand(self, matriz_fire, matriz_pared):
        for nx, ny, _ in self.get_neighbors(matriz_pared):
            vecino = matriz_fire[nx][ny]
            if vecino.state == Fire.SMOKE:
                vecino.state = Fire.FIRE
            elif vecino.state == Fire.FIRE:
                self.to_explode = True  # marcar explosión

        # Propagación de humo aleatorio
        self.set_smoke(matriz_fire, matriz_pared)

    def propagate_explosion(self, matriz_fire, matriz_pared, model):
        for nx, ny, side in self.get_neighbors(matriz_pared):
            vecino = matriz_fire[nx][ny]
            # Poner fuego en celdas vacías o con humo
            if vecino.state == Fire.SMOKE:
                vecino.state = Fire.FIRE
            elif vecino.state == Fire.EMPTY:
                vecino.state = Fire.SMOKE  # marcar explosión
            # Dañar paredes
            if matriz_pared[nx][ny].get_positions(side) == 1:
                matriz_pared[nx][ny].set_damage(side, 0.5)
                model.total_damage += 1
        self.to_explode = False


###############################################################################

height, width = 6, 8
matriz_fire = [[Fire(i,j) for j in range(width)] for i in range(height)]

for (x,y) in fire_coords:
  matriz_fire[x][y].set_state(2)

###############################################################################
# Clase Victim or False Alarm
###############################################################################
class POI:
    VICTIM = "v"
    FALSE = "f"

    def __init__(self, x, y, role):
        self.x = x
        self.y = y
        self.role = role   # "v" o "f"
        self.revealed = False

    def reveal(self):
        """Cuando un bombero llega a la celda se revela si es víctima o falsa alarma."""
        self.revealed = True

    def is_victim(self):
        return self.role == POI.VICTIM and self.revealed

    def is_false(self):
        return self.role == POI.FALSE and self.revealed

    def get_position(self):
        return (self.x, self.y)


###############################################################################
# Corrección de matriz de POIs
###############################################################################

matriz_POI = [[None for j in range(width)] for i in range(height)]
for (x, y, role) in false_alarm_coords:   # viene de parse_positions_v_f
    matriz_POI[x][y] = POI(x, y, role)

[1, 0, 1, 1] [1, 1, 1, 0] [1, 0, 1, 1] [1, 0, 1, 0] [1, 0, 0, 0] [1, 1, 0, 0] [1, 0, 0, 1] [1, 1, 0, 0] 
[1, 0, 0, 0] [1, 0, 0, 0] [1, 0, 1, 0] [1, 1, 1, 0] [0, 0, 1, 1] [0, 1, 1, 0] [0, 0, 1, 1] [0, 1, 1, 0] 
[0, 0, 1, 1] [0, 1, 1, 0] [1, 1, 0, 1] [1, 1, 0, 1] [1, 1, 1, 1] [1, 0, 1, 1] [1, 1, 1, 0] [1, 1, 0, 1] 
[1, 0, 1, 1] [1, 0, 1, 0] [0, 1, 1, 0] [0, 1, 1, 1] [1, 0, 1, 1] [1, 0, 1, 0] [1, 0, 1, 0] [0, 1, 1, 0] 
[1, 0, 0, 1] [1, 1, 1, 0] [1, 0, 0, 1] [1, 1, 0, 0] [1, 1, 0, 1] [1, 1, 0, 1] [1, 1, 0, 1] [1, 1, 0, 1] 
[0, 1, 1, 1] [1, 0, 1, 1] [0, 0, 1, 0] [0, 1, 1, 0] [0, 1, 1, 1] [0, 1, 1, 1] [0, 1, 1, 1] [0, 1, 1, 1] 
Puertas por celda:
Celda (0, 1): 1 puertas
Celda (0, 2): 1 puertas
Celda (0, 5): 1 puertas
Celda (0, 6): 1 puertas
Celda (1, 3): 1 puertas
Celda (1, 4): 1 puertas
Celda (1, 7): 1 puertas
Celda (2, 7): 1 puertas
Celda (2, 2): 1 puertas
Celda (2, 3): 2 puertas
Celda (2, 4): 2 puertas
Celda (2, 5): 2 puertas
Celda (3, 5): 2 puertas
Celda (3, 3): 1 puertas
Celda (4, 3): 1

In [ ]:

# ------------------------------
# Diccionario de direcciones
# ------------------------------
DIRECTIONS = {
    "up":    (1, 0, 0),  # fila-1, col, lado 0
    "left":  (0, -1, 1),  # fila, col-1, lado 1
    "down":  (-1, 0, 2),   # fila+1, col, lado 2
    "right": (0, 1, 3)    # fila, col+1, lado 3
}

# ------------------------------
# Agente con AP
# ------------------------------
class RanaAgent(Agent):
    def __init__(self, model):
        super().__init__(model)
        self.Initial_AP = 4
        self.AP = self.Initial_AP
        self.carry = False

    def sum_AP(self):
        """Recargar AP al inicio del turno"""
        if self.AP == 8:
          return self.AP
        else:
          self.AP += self.Initial_AP
          return self.AP

    def can_move(self, direction, matriz_pared):
        """Verifica si se puede mover en la dirección dada"""
        i, j = self.pos
        di, dj, side = DIRECTIONS[direction]
        ni, nj = i + di, j + dj

      # convertir de (x,y) → (fila, col)
        row, col = i, j
        nrow, ncol = ni, nj

        # 1. Revisar límites
        if nrow < 0 or nrow >= len(matriz_pared) or ncol < 0 or ncol >= len(matriz_pared[0]):
            return False

        current_cell = matriz_pared[row][col]
        neighbor_cell = matriz_pared[nrow][ncol]

        # 2. Revisar paredes
        if current_cell.get_positions(side) == 1:
            return False
        if neighbor_cell.get_positions((side + 2) % 4) == 1:
            return False

        cell1 = (i, j)
        cell2 = (ni, nj)
        for door in cell_doors.get(cell1, []):
            if door.connects(cell1) == cell2:
              if door.blocks_passage():
                return False

        return True

    def get_valid_moves(self, matriz_pared):
      valid_moves = []
      for direction in DIRECTIONS:
          if self.can_move(direction, matriz_pared):
              di, dj, dir = DIRECTIONS[direction]
              ni, nj = self.pos[0] + di, self.pos[1] + dj
              valid_moves.append((ni, nj, dir))
      return valid_moves

    def move(self, matriz_pared, matriz_fire, matriz_POI):
      moves = self.get_valid_moves(matriz_pared)
      if moves:
        new_pos = random.choice(moves)
        (x,y) = new_pos
        self.model.grid.move_agent(self, new_pos)
        self.AP -= 1

        if (x,y) == matriz_fire[x][y] and self.AP >= 2:
          self.AP -= 2
          self.model.grid.move.agent(self,new_pos)

        #destruir paredes
        cell = matriz_pared[x][y]
        if cell.get_positions(dir) == 1 and self.AP > 2:
          cell[x,y].set_damages(0.5)
          self.model.total_damage += 1
          print("Daño total del edificio por rana:", self.model.total_damage)
          self.AP -= 2
          return

        #apagar fuego
        fire_cell = matriz_fire[x][y]
        if fire_cell.get_state() == Fire.FIRE and self.AP > 2:
          matriz_fire[x][y].set_state(Fire.EMPTY)
          self.AP -= 2

        poi = self.model.poi_grid[x][y]
        if poi and not poi.revealed:
          poi.reveal()
          if poi.is_victim() and self.carry == False:
              self.carry = True
              self.model.rescued += 1
              if self.model.rescued >= self.model.rescue_goal:
                    print("VICTORIA: suficientes víctimas rescatadas")
          elif poi.is_false():
                pass  # solo se descarta

        self.sum_AP()


    def step(self):
      self.move(matriz_pared, matriz_fire, matriz_POI)
      


class RanaModel(Model):
    def __init__(self, width=8, height=6, agents=6):
        super().__init__()
        self.grid = MultiGrid(width, height, torus=False)
        self.schedule = RandomActivation(self)

        self.total_damage = 0
        self.max_damage = 24
        self.rescued = 0
        self.lost = 0
        self.rescue_goal = 7
        self.lose_limit = 4
        self.collapsed = False   # <-- bandera para colapso
        self.datacollector = DataCollector(model_reporters={"Grid": get_grid})

        for _ in door_list:
            open_door(access_coords)

        for (x, y) in access_coords:
            doors = cell_doors.get((x, y), [])
            if any(door.get_state() == Door.OPEN for door in doors):
                agent = RanaAgent(self)
                if 0 <= x < width and 0 <= y < height:
                    self.grid.place_agent(agent, (x, y))
                    self.schedule.add(agent)

        self.fire_grid = matriz_fire
        self.poi_grid = matriz_POI

    def step(self):
        if self.collapsed:
            return  # detener el modelo si ya colapsó
#print en daño fuego y daño agente
        # Acciones de agentes
        self.schedule.step()

                # Revisar colapso del edificio
        if self.total_damage >= self.max_damage and not self.collapsed:
            print("DERROTA: edificio colapsado")
            self.collapsed = True
            return

        # Revisar victoria
        if self.rescued >= self.rescue_goal:
            print("VICTORIA: suficientes víctimas rescatadas")
            return

        # Recolectar datos
        self.datacollector.collect(self)


        # Avance del fuego
        for fila in self.fire_grid:
            for celda in fila:
                if celda.state == Fire.FIRE:
                    celda.expand(self.fire_grid, matriz_pared)

        # Propagar explosiones
        for fila in self.fire_grid:
            for celda in fila:
                if celda.to_explode:
                    celda.propagate_explosion(self.fire_grid, matriz_pared, self)

        # Flashover (humo)
        for fila in self.fire_grid:
            for celda in fila:
                celda.set_smoke(self.fire_grid, matriz_pared)
        

        # Revisar víctimas alcanzadas por fuego
        for i in range(len(self.fire_grid)):
            for j in range(len(self.fire_grid[0])):
                poi = self.poi_grid[i][j]
                if self.fire_grid[i][j].state == Fire.FIRE and poi and poi.role == POI.VICTIM:
                    self.lost += 1
                    self.poi_grid[i][j] = None
                    if self.lost >= self.lose_limit:
                        print("DERROTA: demasiadas víctimas perdidas")
                        return



def get_grid(model):
    """Convierte el grid de Mesa en una matriz numpy para análisis o visualización"""
    grid = np.zeros((model.grid.height, model.grid.width))
    for (content, (x, y)) in model.grid.coord_iter():
        if len(content) > 0:
            grid[y][x] = 1   # marcar agente
    return grid




In [62]:
# ------------------------------
# Ejecutar y Animar
# ------------------------------
width=8
height=6
agents=6
model = RanaModel(width, height, agents)
for i in range(1000):
  print(f"Paso {i}")
  model.step()


Paso 0
Paso 1
DERROTA: demasiadas víctimas perdidas
Paso 2
DERROTA: demasiadas víctimas perdidas
Paso 3
DERROTA: edificio colapsado
Paso 4
Paso 5
Paso 6
Paso 7
Paso 8
Paso 9
Paso 10
Paso 11
Paso 12
Paso 13
Paso 14
Paso 15
Paso 16
Paso 17
Paso 18
Paso 19
Paso 20
Paso 21
Paso 22
Paso 23
Paso 24
Paso 25
Paso 26
Paso 27
Paso 28
Paso 29
Paso 30
Paso 31
Paso 32
Paso 33
Paso 34
Paso 35
Paso 36
Paso 37
Paso 38
Paso 39
Paso 40
Paso 41
Paso 42
Paso 43
Paso 44
Paso 45
Paso 46
Paso 47
Paso 48
Paso 49
Paso 50
Paso 51
Paso 52
Paso 53
Paso 54
Paso 55
Paso 56
Paso 57
Paso 58
Paso 59
Paso 60
Paso 61
Paso 62
Paso 63
Paso 64
Paso 65
Paso 66
Paso 67
Paso 68
Paso 69
Paso 70
Paso 71
Paso 72
Paso 73
Paso 74
Paso 75
Paso 76
Paso 77
Paso 78
Paso 79
Paso 80
Paso 81
Paso 82
Paso 83
Paso 84
Paso 85
Paso 86
Paso 87
Paso 88
Paso 89
Paso 90
Paso 91
Paso 92
Paso 93
Paso 94
Paso 95
Paso 96
Paso 97
Paso 98
Paso 99
Paso 100
Paso 101
Paso 102
Paso 103
Paso 104
Paso 105
Paso 106
Paso 107
Paso 108
Paso 109
Paso 110
Paso 11

C:\Users\mbajo\AppData\Local\Temp\ipykernel_5536\1095967249.py:121: DeprecationWarning: The time module and all its Schedulers are deprecated and will be removed in a future version. They can be replaced with AgentSet functionality. See the migration guide for details. https://mesa.readthedocs.io/latest/migration_guide.html#time-and-schedulers
  self.schedule = RandomActivation(self)


In [ ]:
#Conexion local a C#
#Código creado cguiado mediante IA

HOST = "127.0.0.1"
PORT = 65432

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((HOST, PORT))
server.listen()
print(f"Servidor escuchando en {HOST}:{PORT}...")

conn, addr = server.accept()
print(f"Cliente conectado desde {addr}")

ITERATIONS = 30
model = RanaModel()

for i in range(ITERATIONS):
    model.step()
    grid = get_grid(model).tolist()   # importante: convertir numpy → lista

    data = {
        "step": i,
        "grid": grid,
        "agent_pos": (-0.05, 0.05)
    }


    message = json.dumps(data)
    conn.sendall((message + "\n").encode("utf-8"))
    print(f"Enviado paso {i}")

conn.close()
server.close()

In [ ]:
anim